In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from data import *
from vimco import *
from plots import *
from torch.distributions.dirichlet import Dirichlet
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [8]:
T = 30
K = 3
D = 2

## Model Parameters
num_samples = 20
mcmc_steps = 4
num_particles_smc = 100
NUM_HIDDEN = 64
NUM_LATENTS = K*K
NUM_OBS = 2 * K
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-4
CUDA = False

In [9]:
Ys = np.load('dataset/sequences.npy')
As_true = torch.from_numpy(np.load('dataset/transitions.npy')).float()
Zs_true = np.load('dataset/states.npy')
mu_ks = torch.from_numpy(np.load('dataset/means.npy')).float()
cov_ks = torch.from_numpy(np.load('dataset/covariances.npy')).float()
Pi = torch.from_numpy(np.load('dataset/init.npy')).float()

prior = initial_trans_prior(K)

Ys_Zs = np.concatenate((Ys, Zs_true), axis=-1)

In [10]:
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.latent_dir = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, prior, batch_size):
        As = torch.zeros((batch_size, K, K))
        hidden = self.enc_hidden(obs)
        alphas = F.softmax(self.latent_dir(hidden), -1).view(batch_size, T-1, K*K).sum(1).view(batch_size, K, K) + prior
        for k in range(K):
            As[:, k, :] = Dirichlet(alphas[:, k, :]).sample()
        return alphas, As

In [11]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer
enc, optimizer = initialize()

In [12]:
ELBOs = []
KLs = []
Grads = []

prior_mcmc = torch.ones((K, K)).float()
for k in range(K):
    prior_mcmc[k, k] = 2.0
    
Grad_Steps = int((Ys.shape[0] / BATCH_SIZE))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(Ys_Zs)
    Ys_shuffled = torch.from_numpy(Ys_Zs[:, :, :2]).float()
    Zs_true_shuffled = torch.from_numpy(Ys_Zs[:, :, 2:]).float()
    for step in range(Grad_Steps):
        time_start = time.time()
        optimizer.zero_grad()
        batch_data = Ys_shuffled[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        batch_zs = Zs_true_shuffled[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        batch_As = As_true[:BATCH_SIZE]
        gradient, elbo, kl = ag_mcmc_vimco(enc, prior_mcmc, batch_zs, Pi, mu_ks, cov_ks, batch_data, T, D, K, num_samples, num_particles_smc, mcmc_steps, BATCH_SIZE)
        loss = - gradient
        loss.backward()
        optimizer.step()
        ELBOs.append(elbo.item())

        KLs.append(kl.item())
        Grads.append(loss.item())
        time_end = time.time()
        print('epoch : %d, step : %d, ELBO : %f, KL : %f (%ds)' % (epoch, step, elbo, kl, time_end - time_start))

epoch : 0, step : 0, ELBO : -133.393280, KL : 13.039495 (78s)
epoch : 0, step : 1, ELBO : -133.477646, KL : 12.385826 (76s)
epoch : 0, step : 2, ELBO : -134.575485, KL : 11.939428 (76s)
epoch : 0, step : 3, ELBO : -131.904221, KL : 13.581358 (78s)
epoch : 0, step : 4, ELBO : -134.240509, KL : 11.327184 (77s)
epoch : 0, step : 5, ELBO : -133.349380, KL : 11.517175 (78s)
epoch : 0, step : 6, ELBO : -133.213943, KL : 11.093603 (79s)


KeyboardInterrupt: 

In [ ]:
Z_pairs_true = flatz(batch_zs, T, K, BATCH_SIZE)

In [ ]:
variational_new, As_notusing = enc(Z_pairs_true, prior_mcmc, BATCH_SIZE)

In [ ]:
hidden = enc.enc_hidden(Z_pairs_true)
alphas = F.softmax(enc.latent_dir(hidden), -1).view(BATCH_SIZE, T-1, K*K).sum(1).view(BATCH_SIZE, K, K) + prior

In [ ]:
Dirichlet(alphas[:, 0, :]).sample()